<a href="https://colab.research.google.com/github/peravali810/FinetuningLLMs/blob/main/Finetuning_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl

#Step 1: Installing required packages

#Step 2: Importing all installed libraries

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    logging,
    HfArgumentParser,
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

#Step 3: Gathering Model, Dataset and Parameters

In [ ]:
#Selected model from huggingface
mode_name = 'NousResearch/Llama-2-7b-chat-hf'

#Selected Dataset
dataset_name = 'timdettmers/openassistant-guanaco'

#final model name
new_model = 'PJ-Llama-2-7b-chat'

#QLoRA parameters
lora_r = 64 #attention dimension
lora_alpha = 16 #Alpha param for LoRA scaling
lora_dropout = 0.1 #Dropout pobability of layers

#bitsandbytes parameters
use_4bit = True #Activate 4-bit precision base model
bnb_4bit_compute_dtype = 'float16' #Computer dtype for 4-bit base models
bnb_4bit_quant_type = 'nf4' #Quantization type
use_nested_quant = False #Use nested quantization

#Training  Argument parameters

output_dir = './results'
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 3
per_device_eval_batch_size = 3
gradient_accumulation_steps = 2
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = 'paged_adamw_32bit'
lr_scheduler_type = 'cosine'
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

#SFT Parameters
device_map = {"":0}


In [ ]:
torch.cuda.empty_cache()

#Step 4: Loading all and start Fiine tuning process


In [ ]:
#Loading dataset
dataset = load_dataset(dataset_name, split='train')

#Loading tokenizer and model with QLoRA config
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

#checking gpu compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
  major, _ = torch.cuda.get_device_capability()
  if major >= 8:
    print('='* 80)
    print('Your GPU supports bfloat16: accelerate training with bf16=True')
    print('='* 80)

#loading base model
model = AutoModelForCausalLM.from_pretrained(
    mode_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1


#Loading LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_name,trust_remote_code=True)
tokenizer.pad_taken = tokenizer.eos_token_id
tokenizer.padding_side = 'right'


#Loading LoRA Config
peft_config =  LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias='none',
    task_type='CAUSAL_LM'
)

#Set Training param
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)

#Setting supervised finetuning param

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments
)

#Train model
trainer.train()

#Save trained model
trainer.model.save_pretrained(new_model)
